## To use this slideshow:
- Run All, using the menu item: Kernel/Restart & Run All
- Return to this top cell
- click on "Slideshow" menu item above, that looks like this:
![](images/SlideIcon.png)

## Mathematical Modeling

### August 4, 2020 with Laura G Funderburk  

## Session II

In this session, we’ll implement the “<b>S</b>usceptible, <b>E</b>xposed, <b>I</b>nfected and <b>R</b>ecovered” (<b>SEIR</b>) model used in epidemiology, the study of how disease occurs in populations. 


## Recap: What is a Mathematical Model

A mathematical model is a description of a system using <b>mathematical concepts</b> and <b>mathematical language</b>.

You can think of a math model as a tool to help us describe what we believe about the workings of phenomena in the world. 

<b>We use the language of mathematics to express our beliefs.</b>

<b>We use mathematics (theoretical and numerical analysis) to evaluate the model, and get insights about the original phenomenon.</b>

### Building Models: Our Road Map for The Course

|Topic | Session |
|-|-|
|<font color=#000000><b>Choose what phenomenon you want to model|1|</b></font>
|<font color=#000000><b>What assumptions are you making about the phenomenon|1|</b></font>   
|<font color=#000000><b>Use a flow diagram to help you determine the structure of your model|1|</b></font>
|<font color=#1f78b4><b>Choose equations|2|</b></font>
|<font color=#1f78b4><b>Compare against data related to the phenomenon your are modelling|2|</b></font>
|<font color=#1f78b4><b>Solve equations|2|</b></font>
|<font color=#000000><b>Study the behaviour of the model|3|</b></font>
|<font color=#000000><b>Test the model|3|</b></font>
|<font color=#000000><b>Use the model|3|</b></font>


## Recap: Our assumptions

1. Mode of transmission is either 

    a. <b>Primary:</b> the disease is passed from person to person 
    ("contact transmission")

    b. <b>Indirect:</b> the disease is passed via <em>fomites</em> ("non-contact transmission")
    - for example clothes, utensils, and furniture that an infected person coughs on or touches after having touched their nose or mouth 
    
    
2. People can be exposed to the pathogen, get infected, and then recover. We assume people who recovered become susceptible to get reinfected after recovering. 
    

## Our assumptions (continued)

3. There are two ways a population can behave: lockdown (reduced and removal of regular activities) vs non-lockdown (all activies are done normally). 

4. Population is not-constant (that is, people are born and die as time goes by).

5. A person in the population is either one of:
    - susceptible, i.e. not infected but not yet exposed, 
    - exposed to the infection, i.e. is in contact with the virus, but not yet infected, 
    - infected, and 
    - recovered from the infection. 

## Recap: Flow diagram

How does a person move from one stage into another? In other words, how does a person go from susceptible to exposed, to infected, to recovered? 

We can express this change as a rate - a number quantifying how many people moved from one stage to another. Given we do not know what this number is, we will use mathematical symbols to represent this change:

- $\beta, \mu_0$ for the contact and non-contact transmission, respectively, 
- $\kappa$ for exposed to infected,
- $\gamma$ for infected to recovered,
- $\mu _1$ death rate (infected people who died), and
- $\mu _2$ for recovered to susceptible.

## Flow diagram

We can then add the following to our diagram:

$$\text{Susceptible} \stackrel{\beta, \mu_0}{\longrightarrow} \text{Exposed}    $$
$$\mu_2 \uparrow \hspace{2cm} \downarrow \kappa$$
$$\text{Recovered}  \stackrel{\gamma}{\longleftarrow}  \text{Infected}$$
$$\hspace{3.1cm} \downarrow \mu_1$$

$$N(t) = S(t) + E(t) + I(t) + R(t)$$

$$S(t) = −\beta S(t) \frac{I(t)}{N(t)} + \mu_2 R(t) \mu_0 E(t)$$

$$E(t) = \beta S(t) \frac{I(t)}{N(t)}- \kappa E(t)+\mu_0 E$$

$$I(t) = \kappa E(t) − (\gamma + \mu_1) I(t)$$

$$R(t) = \gamma I(t) − \mu_2 R(t)$$

In [ ]:
import numpy as np
import scipy as sp
from scipy.integrate import odeint 
import matplotlib.pyplot as plt

# beta := [0.12, 0.5]
# kappa: = [0.2, 0.3]
# gamma:= [0.25, 0.35]
# mu1 :=[0.025, 0.05]
# mu2: = [0, 0.005]
# μ0: = [0, 0.1]

def ode_model(R0_in=2, De_in=4, Di_in=8):
    def model(z_flat, t):
        z = np.reshape(z_flat, newshape = (4, M))
        s, e, i, r = [z[0, :], z[1, :], z[2, :], z[3, :]]
        n = np.sum(z, axis = 0)
        R0 = R0_in #g_params['R0']
        De = De_in #g_params['De']
        Di = Di_in #g_params['Di']

        ds = -s/n*(R0/Di*i) + (n/n0)*(np.matmul(np.transpose(L), s/n) - s/n*np.matmul(L, np.ones(M)))
        de = s/n*(R0/Di*i) - e/De + (n/n0)*(np.matmul(np.transpose(L), e/n) - e/n*np.matmul(L, np.ones(M)))
        di = e/De - i/Di + (n/n0)*(np.matmul(np.transpose(L), i/n) - i/n*np.matmul(L, np.ones(M)))
        dr = i/Di + (n/n0)*(np.matmul(np.transpose(L), r/n) - r/n*np.matmul(L, np.ones(M)))
        dz = np.array([ds, de, di, dr])
        return np.reshape(dz, newshape = (1, 4*M))[0]
    
    # consider M countries 
    M = 3
    n0 = 100
    c_params = [{''} for i in range(0, M)] # country-specific parameters
    #g_params = {'R0': 2, 'De': 4, 'Di': 8} # global model parameters
    L = np.zeros((M, M)) # matrix of travel rates; L(i, j) = rate of travel from country i --> country j

    s0 = np.array([100, ]*M)
    e0 = np.zeros(M)
    i0 = np.zeros(M)
    i0[0] = 1
    r0 = np.zeros(M)
    z0 = np.array([s0, e0, i0, r0]) # z = [s e i r]

    n = 2000
    t = np.linspace(0, 500, n)
    # solve ODE

    S = np.zeros((n, M))
    E = np.zeros((n, M))
    I = np.zeros((n, M))
    R = np.zeros((n, M))

    S[0] = z0[0, :]
    E[0] = z0[1, :]
    I[0] = z0[2, :]
    R[0] = z0[3, :]
        
    z0 = np.reshape(np.array([s0, e0, i0, r0]), newshape = (1, 4*M))[0] # z = [s e i r]

    for i in range(1,n):
        # span for next time step
        tspan = [t[i-1],t[i]]
        # solve for next step
        z = odeint(model,z0,tspan)[1]
        z = np.reshape(z, newshape = (4, M))
        
        S[i] = z[0, :]
        E[i] = z[1, :]
        I[i] = z[2, :]
        R[i] = z[3, :]

        z0 = np.reshape(z, newshape = (1, 4*M))[0]

    return t, S, I, E

## Session II Take Away

In this session we learned that:

1. 

## Further reading 

https://people.maths.bris.ac.uk/~madjl/course_text.pdf

## Icons 

<a href="https://iconscout.com/icons/thunder" target="_blank">Thunder Icon</a> by <a href="https://iconscout.com/contributors/dinosoftlabs">Dinosoft Lab</a> on <a href="https://iconscout.com">Iconscout</a>


Girl Thinking by <a href="https://www.clipart.email/">clipart.email</a>